In [2]:
import pandas as pd

In [3]:
data_path = '../../data/3.1_temporally_updated_data.feather'
data = pd.read_feather(data_path)#, columns = ['ADR', 'ArrivalDate', 'DepartureDate', 'ReservationStatus'])
data

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,...,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName
0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,...,45,21,45,21,22,44,27,3,Wednesday,July
1,0,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,45,21,43,23,52,63,27,3,Wednesday,July
2,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,45,21,43,23,52,63,27,3,Wednesday,July
3,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,45,21,43,23,52,63,27,3,Wednesday,July
4,0,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,45,21,43,23,52,63,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,2,0.0,0,HB,GBR,Online TA,TA/TO,0,0,...,35,16,28,23,26,14,35,4,Thursday,August
119386,0,2,1.0,0,HB,ITA,Online TA,TA/TO,0,0,...,35,16,28,23,55,48,35,4,Thursday,August
119387,1,2,0.0,0,HB,ESP,Online TA,TA/TO,0,0,...,35,16,32,19,1,60,35,4,Thursday,August
119388,0,2,0.0,0,HB,GBR,Offline TA/TO,TA/TO,0,0,...,35,16,28,23,52,51,35,4,Thursday,August


In [4]:
def explode_reservations(df, start_date_col, end_date_col):
    """
    Explodes reservations in a DataFrame to individual days.

    Args:
        df (pandas.DataFrame): The DataFrame containing the reservations.
        start_date_col (str): The name of the column in `df` that represents the start date of each reservation.
        end_date_col (str): The name of the column in `df` that represents the end date of each reservation.

    Returns:
        pandas.DataFrame: The DataFrame with each reservation expanded to individual days.

    Raises:
        None

    Example:
        df = pd.DataFrame({'check_in': ['2022-01-01', '2022-02-01'],
                           'check_out': ['2022-01-04', '2022-02-03']})
        explode_reservations(df, 'check_in', 'check_out')

    """
    # Create a DataFrame with each guest's stay expanded to individual days
    # Generate a range of dates for each row and store it in a new column
    df['date_range'] = df.apply(lambda row: pd.date_range(start=row[start_date_col],
                                                          end=row[end_date_col] - pd.Timedelta(days=1)),
                                axis=1)

    # Explode the date_range column to create a row for each date in the range
    expanded_df = df.explode('date_range')

    # Drop the original check_in and check_out columns if they are no longer needed
    expanded_df = expanded_df.drop(columns=[start_date_col,
                                            end_date_col])

    # Rename the date_range column to date for clarity
    expanded_df = (expanded_df
                   .rename(columns={'date_range': 'Stay Date'})
                   .reset_index(drop=True)
                   .dropna(subset = 'Stay Date')
                   .sort_values(by = ['Stay Date', 'ADR']))

    # Reset index if needed
    expanded_df = expanded_df

    return expanded_df

In [5]:
data_exploded = explode_reservations(data, 'ArrivalDate', 'DepartureDate')
data_exploded

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,...,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName,Stay Date
164,0,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,...,21,43,23,51,15,27,3,Wednesday,July,2015-07-01
163,0,1,0.0,0,BB,None,Online TA,TA/TO,0,0,...,21,44,22,47,19,27,3,Wednesday,July,2015-07-01
135,0,2,0.0,0,BB,IRL,Offline TA/TO,TA/TO,0,0,...,21,38,28,60,32,27,3,Wednesday,July,2015-07-01
227,0,1,0.0,0,BB,None,Direct,Direct,0,0,...,21,31,35,29,63,27,3,Wednesday,July,2015-07-01
154,0,2,0.0,0,BB,IRL,Offline TA/TO,TA/TO,0,0,...,21,41,25,37,55,27,3,Wednesday,July,2015-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409000,0,2,0.0,0,BB,IRL,Direct,Direct,0,0,...,14,23,28,67,36,35,2,Tuesday,August,2017-09-11
409699,0,2,0.0,0,HB,DEU,Offline TA/TO,TA/TO,0,0,...,16,21,30,22,81,35,4,Thursday,August,2017-09-12
409822,0,2,0.0,0,HB,GBR,Offline TA/TO,TA/TO,0,0,...,16,21,30,72,31,35,4,Thursday,August,2017-09-12
409700,0,2,0.0,0,HB,DEU,Offline TA/TO,TA/TO,0,0,...,16,21,30,22,81,35,4,Thursday,August,2017-09-13


In [6]:
data_exploded.to_feather('../../data/3.3_data_exploded.feather')